# Industry 4.0 의 중심, AI - ML&DL

<div align='right'><font size=2 color='gray'>Machine Learning & Deep Learning with TensorFlow @ <font color='blue'><a href='https://www.facebook.com/jskim.kr'>FB / jskim.kr</a></font>, 김진수</font></div>
<hr>

# Sect12. RNN (Recurrent Newural Network)

In [1]:
from images import bigpycraft_ai as bpc
from IPython.display import Image 
from tqdm import tqdm_notebook

In [2]:
import time
import os

def chk_processting_time(start_time, end_time):
    process_time = end_time - start_time
    p_time = int(process_time)
    p_min = p_time // 60
    p_sec = p_time %  60
    print('처리시간 : {p_min}분 {p_sec}초 경과되었습니다.'.format(
            p_min = p_min, 
            p_sec = p_sec
        ))
    return process_time

### 12-3. only_softmax_char_seq

<hr>
``` python 

# Lab 12 Character Sequence Softmax only
import tensorflow as tf
import numpy as np
tf.set_random_seed(777)  # reproducibility

sample = " if you want you"
idx2char = list(set(sample))  # index -> char
char2idx = {c: i for i, c in enumerate(idx2char)}  # char -> idex

# hyper parameters
dic_size = len(char2idx)  # RNN input size (one hot size)
rnn_hidden_size = len(char2idx)  # RNN output size
num_classes = len(char2idx)  # final output size (RNN or softmax, etc.)
batch_size = 1  # one sample data, one batch
sequence_length = len(sample) - 1  # number of lstm rollings (unit #)

sample_idx = [char2idx[c] for c in sample]  # char to index
x_data = [sample_idx[:-1]]  # X data sample (0 ~ n-1) hello: hell
y_data = [sample_idx[1:]]   # Y label sample (1 ~ n) hello: ello

X = tf.placeholder(tf.int32, [None, sequence_length])  # X data
Y = tf.placeholder(tf.int32, [None, sequence_length])  # Y label

# flatten the data (ignore batches for now). No effect if the batch size is 1
X_one_hot = tf.one_hot(X, num_classes)  # one hot: 1 -> 0 1 0 0 0 0 0 0 0 0
X_for_softmax = tf.reshape(X_one_hot, [-1, rnn_hidden_size])

# softmax layer (rnn_hidden_size -> num_classes)
softmax_w = tf.get_variable("softmax_w", [rnn_hidden_size, num_classes])
softmax_b = tf.get_variable("softmax_b", [num_classes])
outputs = tf.matmul(X_for_softmax, softmax_w) + softmax_b

# expend the data (revive the batches)
outputs = tf.reshape(outputs, [batch_size, sequence_length, num_classes])
weights = tf.ones([batch_size, sequence_length])

# Compute sequence cost/loss
sequence_loss = tf.contrib.seq2seq.sequence_loss(
    logits=outputs, targets=Y, weights=weights)
loss = tf.reduce_mean(sequence_loss)  # mean all sequence loss
train = tf.train.AdamOptimizer(learning_rate=0.1).minimize(loss)

prediction = tf.argmax(outputs, axis=2)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for t_cnt in range(3000):
        l, _ = sess.run([loss, train], feed_dict={X: x_data, Y: y_data})
        result = sess.run(prediction, feed_dict={X: x_data})

        # print char using dic
        result_str = [idx2char[c] for c in np.squeeze(result)]
        print(t_cnt, "loss:", l, "Prediction:", ''.join(result_str))

'''
 1th training... 	 Loss : 2.599400520324707, 	 Prediction : uyuunuiutnffunu 
 2th training... 	 Loss : 2.378079414367676, 	 Prediction : uyuunu utnfuunu 
 3th training... 	 Loss : 2.1724300384521484, 	 Prediction : uyuuuu utntuuuu 
 4th training... 	 Loss : 1.980708122253418, 	 Prediction : iy iou iynt iou 
 5th training... 	 Loss : 1.8024282455444336, 	 Prediction : yy you yynt you 

..
..

 2995th training... 	 Loss : 0.2773257791996002, 	 Prediction : yf you yant you 
 2996th training... 	 Loss : 0.2773257791996002, 	 Prediction : yf you yant you 
 2997th training... 	 Loss : 0.2773257791996002, 	 Prediction : yf you yant you 
 2998th training... 	 Loss : 0.27732571959495544, 	 Prediction : yf you yant you 
 2999th training... 	 Loss : 0.27732568979263306, 	 Prediction : yf you yant you 
 3000th training... 	 Loss : 0.2773256301879883, 	 Prediction : yf you yant you 
 
'''


```

<hr>

In [3]:
# Lab 12 Character Sequence Softmax only
import tensorflow as tf
import numpy as np
tf.set_random_seed(777)  # reproducibility


C:\Python\Anaconda3-50\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [4]:
# sample = " if you want you"
sample   = "만약에 당신이 그 누구와 사랑에 빠지면"
idx2char = list(set(sample))  # index -> char
char2idx = {c: i for i, c in enumerate(idx2char)}  # char -> idex


In [5]:
# hyper parameters
dic_size        = len(char2idx)     # RNN input size (one hot size)
rnn_hidden_size = len(char2idx)     # RNN output size
num_classes     = len(char2idx)     # final output size (RNN or softmax, etc.)
batch_size      = 1                 # one sample data, one batch
sequence_length = len(sample) - 1   # number of lstm rollings (unit #)

In [6]:
sample_idx = [char2idx[c] for c in sample]  # char to index
x_data = [sample_idx[:-1]]    # X data sample (0 ~ n-1) hello: hell
y_data = [sample_idx[1:]]     # Y label sample (1 ~ n) hello: ello

X = tf.placeholder(tf.int32, [None, sequence_length])  # X data
Y = tf.placeholder(tf.int32, [None, sequence_length])  # Y label


In [7]:
# flatten the data (ignore batches for now). No effect if the batch size is 1
X_one_hot = tf.one_hot(X, num_classes)  # one hot: 1 -> 0 1 0 0 0 0 0 0 0 0
X_for_softmax = tf.reshape(X_one_hot, [-1, rnn_hidden_size])

# softmax layer (rnn_hidden_size -> num_classes)
softmax_w = tf.get_variable("softmax_w", [rnn_hidden_size, num_classes])
softmax_b = tf.get_variable("softmax_b", [num_classes])
outputs = tf.matmul(X_for_softmax, softmax_w) + softmax_b

# expend the data (revive the batches)
outputs = tf.reshape(outputs, [batch_size, sequence_length, num_classes])
weights = tf.ones([batch_size, sequence_length])

# Compute sequence cost/loss
sequence_loss = tf.contrib.seq2seq.sequence_loss(
    logits=outputs, targets=Y, weights=weights)
loss = tf.reduce_mean(sequence_loss)  # mean all sequence loss
train = tf.train.AdamOptimizer(learning_rate=0.1).minimize(loss)

prediction = tf.argmax(outputs, axis=2)

In [8]:
time1 = time.time()

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for t_cnt in range(3000):
        l, _ = sess.run([loss, train], feed_dict={X: x_data, Y: y_data})
        result = sess.run(prediction, feed_dict={X: x_data})

        # print char using dic
        result_str = [idx2char[c] for c in np.squeeze(result)]

        cnt = t_cnt+1
        if cnt<20 or cnt%100==0:
            # print(t_cnt, "loss:", l, "Prediction:", ''.join(result_str))
            print(" {cnt}th training... \t Loss : {loss}, \t Prediction : {pred} ".format(
                    cnt  = cnt, 
                    loss = l, 
                    pred = ''.join(result_str)
            ))

time2 = time.time()

 1th training... 	 Loss : 2.81402850151062, 	 Prediction : 면  사약약 사당사당빠당사랑이 사사  
 2th training... 	 Loss : 2.595418691635132, 	 Prediction : 약  사약이 사 사당  사랑구 사사  
 3th training... 	 Loss : 2.388251543045044, 	 Prediction : 약  사신이 사 사   사랑구 사사  
 4th training... 	 Loss : 2.1938159465789795, 	 Prediction : 약  사신이 사 사   사랑에 사에  
 5th training... 	 Loss : 2.0127334594726562, 	 Prediction : 약  사신이 사 사   사랑에 사에  
 6th training... 	 Loss : 1.8459144830703735, 	 Prediction : 약에 사신이 사 사 와 사랑에 사에  
 7th training... 	 Loss : 1.6937185525894165, 	 Prediction : 약에 사신이 사 사구와 사랑에 사에  
 8th training... 	 Loss : 1.5556710958480835, 	 Prediction : 약에 사신이 사 사구와 사랑에 사지  
 9th training... 	 Loss : 1.4307368993759155, 	 Prediction : 약에 사신이 사 사구와 사랑에 사지  
 10th training... 	 Loss : 1.3175714015960693, 	 Prediction : 약에 사신이 사 사구와 사랑에 사지  
 11th training... 	 Loss : 1.214782953262329, 	 Prediction : 약에 사신이 사 사구와 사랑에 사지면 
 12th training... 	 Loss : 1.1211830377578735, 	 Prediction : 약에 사신이 사 사구와 사랑에 사지면 
 13th 

In [9]:
chk_processting_time(time1, time2)

처리시간 : 0분 2초 경과되었습니다.


2.2330777645111084

In [10]:
predict_sentence = sample[0] + ''.join(result_str)
predict_sentence

'만약에 그신이 그 그구와 그랑에 그지면'

<hr>
<marquee><font size=3 color='brown'>The BigpyCraft find the information to design valuable society with Technology & Craft.</font></marquee>
<div align='right'><font size=2 color='gray'> &lt; The End &gt; </font></div>